In [ ]:
!pip install PyDrive

In [1]:
import os
import mykeys

project_name = 'CH16_Agent'

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = project_name
os.environ["LANGCHAIN_API_KEY"] = mykeys.get_key('LANG')
os.environ["LANGCHAIN_HUB_API_KEY"] = mykeys.get_key('LANG')
os.environ["OPENAI_API_KEY"] = mykeys.get_key('GPT')
os.environ["GOOGLE_API_KEY"] = mykeys.get_key('GOO')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = mykeys.get_key('HF')
os.environ["UPSTAGE_API_KEY"] = mykeys.get_key('UP')
os.environ["COHERE_API_KEY"] = mykeys.get_key('COH')
os.environ["JINA_API_KEY"] = mykeys.get_key('JINA')
os.environ["ANTHROPIC_API_KEY"] = mykeys.get_key('ANT')
os.environ["DEEPL_API_KEY"] = mykeys.get_key('DEEP')
os.environ["TAVILY_API_KEY"] = mykeys.get_key('TAV')

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Authentication successful.


In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith(project_name)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16_Agent


In [ ]:
from langchain_teddynote import logging

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith(project_name, set_enable=False)

CH16 에이전트(Agent)

01. 도구(Tools)

In [8]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드를 실행하는 도구를 생성합니다.
python_tool = PythonREPLTool()


In [9]:
# 파이썬 코드를 실행하고 결과를 반환합니다.
print(python_tool.invoke("print(100 + 200)"))


Python REPL can execute arbitrary code. Use with caution.


300



In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


# 파이썬 코드를 실행하고 중간 과정을 출력하고 도구 실행 결과를 반환하는 함수
def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)


# 파이썬 코드를 작성하도록 요청하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)
# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4.1", temperature=0)

# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)


In [ ]:
# 결과 출력
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

CODE:
import random

def generate_lotto():
    return sorted(random.sample(range(1, 46), 6))

if __name__ == "__main__":
    print("로또 번호:", generate_lotto())



In [4]:
!pip install -U langchain-tavily

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced", # or "basic"
    include_domains=["github.io", "wikidocs.net"],
    # exclude_domains = []
)


In [8]:
# 도구 실행
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})


[{'title': 'Tool calling - GitHub Pages',
  'url': 'https://langchain-ai.github.io/langgraph/concepts/tools/',
  'content': '[Skip to content](https://langchain-ai.github.io/langgraph/concepts/tools/#tools) [](https://langchain-ai.github.io/langgraph/concepts/tools/?q= "Share") *   - [x] [Guides](https://langchain-ai.github.io/langgraph/)  Guides   *   [Overview](https://langchain-ai.github.io/langgraph/agents/overview/) *   [Models](https://langchain-ai.github.io/langgraph/agents/models/) *   [Tools](https://langchain-ai.github.io/langgraph/agents/tools/) *   [Agent architectures](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/) *   - [x]  Overview  [Overview](https://langchain-ai.github.io/langgraph/concepts/tools/) Table of contents   *   [Tool calling](https://langchain-ai.github.io/langgraph/concepts/tools/#tool-calling) *   [Execute tools](https://langchain-ai.github.io/langgraph/concepts/tools/#execute-tools) *   [Prebuilt tools](https://langchain-ai.github.i

In [9]:
from langchain_teddynote.tools.tavily import TavilySearch

# 기본 예제
tavily_tool = TavilySearch()

# include_domains 사용 예제
# 특정 도메인만 포함하여 검색
tavily_tool_with_domains = TavilySearch(include_domains=["github.io", "naver.com"])

# exclude_domains 사용 예제
# 특정 도메인을 제외하고 검색
tavily_tool_exclude = TavilySearch(exclude_domains=["ads.com", "spam.com"])

# 다양한 파라미터를 사용한 검색 예제
result1 = tavily_tool.search(
    query="유튜버 테디노트에 대해서 알려줘",  # 검색 쿼리
    search_depth="advanced",  # 고급 검색 수준
    topic="general",  # 일반 주제
    days=7,  # 최근 7일 내 결과
    max_results=10,  # 최대 10개 결과
    include_answer=True,  # 답변 포함
    include_raw_content=True,  # 원본 콘텐츠 포함
    include_images=True,  # 이미지 포함
    format_output=True,  # 결과 포맷팅
)

# 뉴스 검색 예제
result2 = tavily_tool.search(
    query="최신 AI 기술 동향",  # 검색 쿼리
    search_depth="basic",  # 기본 검색 수준
    topic="news",  # 뉴스 주제
    days=3,  # 최근 3일 내 결과
    max_results=5,  # 최대 5개 결과
    include_answer=False,  # 답변 미포함
    include_raw_content=False,  # 원본 콘텐츠 미포함
    include_images=False,  # 이미지 미포함
    format_output=True,  # 결과 포맷팅
)

# 특정 도메인 포함 검색 예제
result3 = tavily_tool_with_domains.search(
    query="파이썬 프로그래밍 팁",  # 검색 쿼리
    search_depth="advanced",  # 고급 검색 수준
    max_results=3,  # 최대 3개 결과
)

# 특정 도메인 제외 검색 예제
result4 = tavily_tool_exclude.search(
    query="건강한 식단",  # 검색 쿼리
    search_depth="basic",  # 기본 검색 수준
    days=30,  # 최근 30일 내 결과
    max_results=7,  # 최대 7개 결과
)

# 결과 출력
print("기본 검색 결과:", result1)
print("뉴스 검색 결과:", result2)
print("특정 도메인 포함 검색 결과:", result3)
print("특정 도메인 제외 검색 결과:", result4)


기본 검색 결과: ['<document><title>테디노트 - 위키독스</title><url>https://wikidocs.net/profile/info/book/10226</url><content>테디노트 - 위키독스\\n\\n===============\\n\\n위키독스\\n\\n   위키\\n   블로그\\n\\n   도서 증정 이벤트\\n   로그인\\n\\n### 테디노트\\n\\n  \\n\\n한줄 소개 블로그와 유튜브 \\"테디노트\\"를 운영하고 있으며, \\"파이썬 딥러닝 텐서플로\\"를 집필하였습니다. 데이터분석과 AI를 사랑하고 지식공유에 활발히 참여하고 있습니다. 첫번째 컴퓨터 기억이 안나요.블로그테디노트나의 URL\\n\\n   저서\\n   최근댓글\\n   최근수정\\n   최근블로그\\n\\n 추천순   인기순   최신순  \\n\\n  \\n\\nImage 1\\n\\n<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷\\n\\n 저자: 테디노트 \\n\\n 추천은 공유할 수 있는 무료 전자책을 집필하는데 정말 큰 힘이 됩니다. \\"추천\\" 한 번씩만 부탁 드리겠습니다🙏🙏 ✅ 랭체인 한국어 튜토리얼 강의 패스트캠퍼스 - RAG ✅ 랭체인 한국어 튜토리얼 코드저장소(GitHub) 📘🖥️ ✅ 유튜브 \\"테디노트\\"… \\n\\n 수정: 2025년 4월 30일 8:06 오후 [...] 한 권으로 끝내는 <머신러닝 노트>\\n\\n 저자: 테디노트 \\n\\n 머신러닝은 최근 가장 주목받고 있는 인공지능 분야 가운데 하나입니다. 머신러닝은 데이터를 기반으로 다양한 통계적 알고리즘을 활용하여 컴퓨터가 스스로 학습할 수 있도록 프로그래밍 하는 것을 의미합니다. 머신러닝 노트는 이러한 머신러닝에 대한 학습을 어디서부터 시작할지 막막해 하는 분들을… \\n\\n 수정: 2024년 2월 2일 2:45 오전 \\n\\nImage 5\\n\\n테디노트의 RAG 비법노트 : RAG 시작하기\\n\\n 저자: 패스트캠퍼스,

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4.1", temperature=0.9, max_tokens=1000)

# DALL-E 이미지 생성을 위한 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# 프롬프트, LLM, 출력 파서를 연결하는 체인 생성
chain = prompt | llm | StrOutputParser()

# 체인 실행
image_prompt = chain.invoke(
    {"image_desc": "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"}
)

# 이미지 프롬프트 출력
print(image_prompt)


A neo-classicism style oil painting depicting a group of people in classical Roman attire, gathered in an elegant marble hall with grand columns and ornate sculptures. Each person is intensely gazing at a modern smartphone held in their hands, their expressions absorbed and detached from one another. The lighting is dramatic, with soft golden sunlight streaming through large arched windows, casting deep shadows and highlighting the meticulous details of their garments and surroundings. The scene satirically contrasts the timeless grandeur of the setting with the contemporary obsession with mobile technology.


In [ ]:
# DALL-E API 래퍼 가져오기
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image

# DALL-E API 래퍼 초기화
# model: 사용할 DALL-E 모델 버전
# size: 생성할 이미지 크기
# quality: 이미지 품질
# n: 생성할 이미지 수
dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)

# 질문
query = "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"

# 이미지 생성 및 URL 받기
# chain.invoke()를 사용하여 이미지 설명을 DALL-E 프롬프트로 변환
# dalle.run()을 사용하여 실제 이미지 생성
image_url = dalle.run(chain.invoke({"image_desc": query}))

# 생성된 이미지를 표시합니다.
Image(url=image_url, width=500)


In [42]:
print("image_url=",image_url)

image_url= https://oaidalleapiprodscus.blob.core.windows.net/private/org-Soffyuxth7vsUnobnzk0MGeR/user-7ZaIktSNwrpSR2x5BHKfLfq8/img-aeVZjTv6ID7SkJybyWgb1ZlL.png?st=2025-08-25T04%3A37%3A34Z&se=2025-08-25T06%3A37%3A34Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=32836cae-d25f-4fe9-827b-1c8c59c442cc&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-08-24T15%3A11%3A57Z&ske=2025-08-25T15%3A11%3A57Z&sks=b&skv=2024-08-04&sig=enYAlSiPrkPhCl1QwhjYD/8yY%2BNlmgfwo1MuKUPWxNk%3D


In [13]:
from langchain.tools import tool


# 데코레이터를 사용하여 함수를 도구로 변환합니다.
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [14]:
# 도구 실행
add_numbers.invoke({"a": 3, "b": 4})


7

In [15]:
# 도구 실행
multiply_numbers.invoke({"a": 3, "b": 4})


12

In [16]:
from langchain_teddynote.tools import GoogleNews

# 도구 생성
news_tool = GoogleNews()


In [17]:
# 최신 뉴스 검색
news_tool.search_latest(k=5)


[{'url': 'https://news.google.com/rss/articles/CBMia0FVX3lxTE10ckt6aWw1YUtpMW9aam5Xc21CUkd5Rzl5YVdzbFoxZ0NJRFNUMFBSWTgyUE5oVVlGY01iY2JWTGROb3kycWR3eG9fXzA2SjZQOE5NSnZjX25kQWxVTi11V2toNlJYODRwVW5V?oc=5',
  'content': '한일회담 마친 이 대통령 “과거사 지적 각오…첫술에 배부르려면 체해” - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMihAFBVV95cUxQemZrdWszNE0zVXhndjJ0Nm5uYmtaVjc5NHo2QlpqWHh2OHBidXpjTzhocnlZS0hHTWRvMTFURjRmejh1SWswY0w5NXFNbUY3M1ZTMWxyYzU2bmw3YThTRWlFUHZqLWtvV2tCbTRFSmtwVWQxcUZSenA3WFBMaUY0QmtYVHbSAZgBQVVfeXFMTVlNZmNTWWtQMXdLYk55eW81S2xRSmQwQXJGTDVsZ2VoSmptX3Mycms0T3lBVjEzZEstUFY3OWxnYUN2N2NadWJFNkV4WUdvR0lUMWdoaUtSUWk1WmFrejNQOXp5VFZkMzBRQlo4UHBrdXVGdVV1c1I5aXMtYklidHNQdm1RTG9ZeXRmcVNwZTU2SWx2blFOVkY?oc=5',
  'content': '전용기서 내린 李대통령, 국무부 부의전장이 맞았다 - 조선일보'},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE1OV1M3eGtfQ2phcUVzeklSWUVYS2Vld0pmOERnVnJ4UkRtVHZOU0hkNzdrT2hkdTFsYWhjeTdEbDFUUE0xcHZhVnNiamNXV2l0cGZpMmI0YWtYQdIBX0FVX3lxTE9mOEUtM24tU1RwYkZuWk9ROG05amNHZi11UV9CU0g0UmliN09kaW

In [18]:
# 키워드로 뉴스 검색
news_tool.search_by_keyword("AI 투자", k=5)


[{'url': 'https://news.google.com/rss/articles/CBMiYEFVX3lxTFBoVHd6ejFhYTBmYm5aZjJ0Rmx0U0J6eGVsYjFxcDh2LV91eWhVTU1EeUFWWHBUZUJHSWRKVVZ1QUFJVTMtazhMbDI2a1hDcEYzTVZZYTNZU0F3cWt6Nlg3etIBeEFVX3lxTFBFWFYxcFhSU2VYMloxMWpiNFZhbFRacXlBMy16OFVURl9YR3pyeFFDczBVT2ZCY0pIMkltTVFtbms5WFNjc3paaHhyWWY2d1BBZ2ZYUlVpNVI1dkdGTEtzRFBQajJvWmg2dF95bXJ6OHlwTWFDSkZVNw?oc=5',
  'content': '"AI 버블? 한국은 정반대…투자 침체에 창업 의지 꺾여"[AI 버블론下] - 뉴시스'},
 {'url': 'https://news.google.com/rss/articles/CBMidkFVX3lxTE0zaDZwN29zeVVBYVkzU3ZHZTF0eXhSRldoMF9SaFN2TTFaMjdwdHR3ZEhrNXdrMzdmYVJQTnpXOUZRbjJiRmNOakN4cGw0ZE1ubFFnV2ZTeTZTcE01QVVCWTJ0ZXlfd05mTldMckRuTjMzRTVDcHfSAWZBVV95cUxPQW5NUnluU290Y0FTRGJmYVM3ME4yamtSWkNMZ3k4VVBTUElBTk1yRk5Wcy1VRS1fQUNVRGszMHhpLXQwTkh1VVdpUmZ4MGRCYmFvbGE3QWEya0xscDNQdGIxNmZsY2c?oc=5',
  'content': '내년 R&D 예산 35.3조 역대 최대…AI 투자 2배 늘린다 - 동아일보'},
 {'url': 'https://news.google.com/rss/articles/CBMif0FVX3lxTE44OUlPbWkwbXVZX2JNaHprMF9Ndi0wcnVZcUJXSjVVaFRaSmFYelRmNXZ2S0NKWWV5YXIzYTdNYUVCelZHZlk4VEU5aHY2cDBGYTRXMj

In [19]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict


# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


In [20]:
# 실행 결과
search_keyword.invoke({"query": "AI 투자"})


AI 투자


[{'url': 'https://news.google.com/rss/articles/CBMiYEFVX3lxTFBoVHd6ejFhYTBmYm5aZjJ0Rmx0U0J6eGVsYjFxcDh2LV91eWhVTU1EeUFWWHBUZUJHSWRKVVZ1QUFJVTMtazhMbDI2a1hDcEYzTVZZYTNZU0F3cWt6Nlg3etIBeEFVX3lxTFBFWFYxcFhSU2VYMloxMWpiNFZhbFRacXlBMy16OFVURl9YR3pyeFFDczBVT2ZCY0pIMkltTVFtbms5WFNjc3paaHhyWWY2d1BBZ2ZYUlVpNVI1dkdGTEtzRFBQajJvWmg2dF95bXJ6OHlwTWFDSkZVNw?oc=5',
  'content': '"AI 버블? 한국은 정반대…투자 침체에 창업 의지 꺾여"[AI 버블론下] - 뉴시스'},
 {'url': 'https://news.google.com/rss/articles/CBMidkFVX3lxTE0zaDZwN29zeVVBYVkzU3ZHZTF0eXhSRldoMF9SaFN2TTFaMjdwdHR3ZEhrNXdrMzdmYVJQTnpXOUZRbjJiRmNOakN4cGw0ZE1ubFFnV2ZTeTZTcE01QVVCWTJ0ZXlfd05mTldMckRuTjMzRTVDcHfSAWZBVV95cUxPQW5NUnluU290Y0FTRGJmYVM3ME4yamtSWkNMZ3k4VVBTUElBTk1yRk5Wcy1VRS1fQUNVRGszMHhpLXQwTkh1VVdpUmZ4MGRCYmFvbGE3QWEya0xscDNQdGIxNmZsY2c?oc=5',
  'content': '내년 R&D 예산 35.3조 역대 최대…AI 투자 2배 늘린다 - 동아일보'},
 {'url': 'https://news.google.com/rss/articles/CBMif0FVX3lxTE44OUlPbWkwbXVZX2JNaHprMF9Ndi0wcnVZcUJXSjVVaFRaSmFYelRmNXZ2S0NKWWV5YXIzYTdNYUVCelZHZlk4VEU5aHY2cDBGYTRXMj

02. 도구 바인딩(Binding Tools)

In [21]:
import re
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool


# 도구를 정의합니다.
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b


@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"


tools = [get_word_length, add_function, naver_news_crawl]


In [22]:
from langchain_openai import ChatOpenAI

# 모델 생성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools)


In [23]:
# 실행 결과
llm_with_tools.invoke("What is the length of the word 'teddynote'?").tool_calls


[{'name': 'get_word_length',
  'args': {'word': 'teddynote'},
  'id': 'call_2ZsORAgl797okuCndALcA8xV',
  'type': 'tool_call'}]

In [24]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# 도구 바인딩 + 도구 파서
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

# 실행 결과
tool_call_results = chain.invoke("What is the length of the word 'teddynote'?")


In [25]:
print(tool_call_results, end="\n\n==========\n\n")
# 첫 번째 도구 호출 결과
single_result = tool_call_results[0]
# 도구 이름
print(single_result["type"])
# 도구 인자
print(single_result["args"])


[{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}]


get_word_length
{'word': 'teddynote'}


In [26]:
tool_call_results[0]["type"], tools[0].name


('get_word_length', 'get_word_length')

In [27]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    # 도구 호출 결과 리스트를 순회합니다.
    for tool_call_result in tool_call_results:
        # 도구의 이름과 인자를 추출합니다.
        tool_name = tool_call_result["type"]
        tool_args = tool_call_result["args"]

        # 도구 이름과 일치하는 도구를 찾아 실행합니다.
        # next() 함수를 사용하여 일치하는 첫 번째 도구를 찾습니다.
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구를 찾았다면 해당 도구를 실행합니다.
            result = matching_tool.invoke(tool_args)
            # 실행 결과를 출력합니다.
            print(f"[실행도구] {tool_name}\n[실행결과] {result}")
        else:
            # 일치하는 도구를 찾지 못했다면 경고 메시지를 출력합니다.
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")


# 도구 호출 실행
# 이전에 얻은 tool_call_results를 인자로 전달하여 함수를 실행합니다.
execute_tool_calls(tool_call_results)


[실행도구] get_word_length
[실행결과] 9


In [28]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# bind_tools + Parser + Execution
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls


In [29]:
# 실행 결과
chain.invoke("What is the length of the word 'teddynote'?")


[실행도구] get_word_length
[실행결과] 9


In [30]:
# 실행 결과
chain.invoke("114.5 + 121.2")
print(114.5 + 121.2)


[실행도구] add_function
[실행결과] 235.7
235.7


In [31]:
# 실행 결과
chain.invoke(
    "뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
)


[실행도구] naver_news_crawl
[실행결과] [미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국채지수(WGBI) 에 편입될 지 관심. 주식의 경우 지난 2009년부터 선진국 지수에 편입돼 있는 한국 증시에 대해 공매도 제한 등을 이유로 관찰 대상국으로 지정할지 관심. 지정되더라도 검토 기간이 있어 즉각 제외되지는 않음.
손희연 기자(kunst@zdnet.co.kr

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Agent프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = ChatOpenAI(model="gpt-4.1", temperature=0)


In [34]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 이전에 정의한 도구 사용
tools = [get_word_length, add_function, naver_news_crawl]

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)


In [35]:
# Agent 실행
result = agent_executor.invoke({"input": "How many letters in the word `teddynote`?"})

# 결과 확인
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'teddynote'}`


9The word "teddynote" has 9 letters.

> Finished chain.
The word "teddynote" has 9 letters.


In [38]:
# Agent 실행
result = agent_executor.invoke({"input": "114.5 + 121.2 의 계산 결과는?"})

# 결과 확인
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7114.5 + 121.2의 계산 결과는 235.7입니다.

> Finished chain.
114.5 + 121.2의 계산 결과는 235.7입니다.


In [39]:
# Agent 실행
result = agent_executor.invoke(
    {"input": "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"}
)

# 결과 확인
print(result["output"])
print("==========\n")
print(114.5 + 121.2 + 34.2 + 110.1)




> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 34.2, 'b': 110.1}`


144.3
Invoking: `add_function` with `{'a': 235.7, 'b': 144.3}`


380.0114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

> Finished chain.
114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

380.0


In [40]:
result = agent_executor.invoke(
    {
        "input": "뉴스 기사를 요약해 줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
    }
)
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}`


[미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국